In [ ]:
#| default_exp deployment


In [1]:
#| export
import datetime
from typing import Dict, Any, List
from snowflake.ml.jobs import remote
from snowflake.ml.registry import registry
from snowflake.snowpark import Session
from snowflake.ml.ray.datasink.table_data_sink import SnowflakeTableDatasink
from snowflake.ml.ray.datasource import SFStageTextDataSource
import ray


ModuleNotFoundError: No module named 'snowflake.ml.ray'

In [2]:
#| export  
def create_update_reviews_job(
        compute_pool: str = "HOL_COMPUTE_POOL_HIGHMEM",
        stage_name: str = "payload_stage",
        integration: str = "ALLOW_ALL_ACCESS_INTEGRATION"
    ):
    """Create a remote job for updating reviews with quality predictions.
    
    Args:
        compute_pool: Name of the compute pool to use
        stage_name: Name of the stage for job artifacts
        integration: Name of the external access integration
        
    Returns:
        Decorated function for remote execution
    """
    @remote(compute_pool=compute_pool, 
            stage_name=stage_name, 
            external_access_integrations=[integration])
    def update_reviews():
        # Import inside function for remote execution
        from SnowflakeML.data_processing import parse_reviews
        from SnowflakeML.ml_models import ModelPredictor
        
        file_name = "*.txt"
        reviews_stage = "REVIEWS"
        
        text_source = SFStageTextDataSource(
            stage_location=reviews_stage,
            file_pattern=file_name
        )
        
        text_dataset = ray.data.read_datasource(text_source)
        parsed_dataset = text_dataset.map(parse_reviews)
        dataset = parsed_dataset.map_batches(
            ModelPredictor, 
            concurrency=1, 
            batch_size=10, 
            num_cpus=24
        )

        datasink = SnowflakeTableDatasink(
            table_name="REVIEWS",
            auto_create_table=True,
            override=False,
        )
        dataset.write_datasink(datasink)
    
    return update_reviews


In [3]:
#| hide
import nbdev; nbdev.nbdev_export()